In [ ]:
%matplotlib inline

# Raw datasets

Accessing raw array data from an experiment.

In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from e11 import H5Data

## hdf5 file

In [ ]:
import os 
fil = os.path.join(os.getcwd(), 'example_data', 'array_data.h5')
# read hdf5 file
data = H5Data(fil)
data.pprint()

In [ ]:
# List the datasets that belong to the group `squid=1`.
print(data.datasets(1))

## Array datasets

### Oscilloscope data

Oscilloscope traces are often stored as 2d arrays of repeating 1d measurements.

In [ ]:
# Load 2D array data from the hdf5 file,
squid = 1
arr = data.array(dataset='OSC_0', squids=squid, ignore_missing=False)

This data contains traces of 2501 data points and 25 repeats are stored along the first dimension.


In [ ]:
print(np.shape(arr))

In [ ]:
# information about the dataset.
osc_info = data.dataset_attrs('OSC_0', squid)
osc_info

In [ ]:
# plot
fig, ax = plt.subplots()

# data
dt = 1e6 * osc_info['dt']
xvals = np.arange(0, len(arr[0])*dt, dt)
yvals = np.mean(arr, axis=0)
ax.plot(xvals, yvals)

# format
ax.set_xlabel('time ($\mu$s)')
ax.set_ylabel('signal (V)')

# output
plt.show()

### Camera data

Camera data are typically stored as 3d arrays of repeating 2d exposures.  Here, the repeats are stored along axis=2, in accordance with the convention of `HDFView`.

In [ ]:
# read image datasets
squids = [1, 2]
img = data.array(dataset='IMG', squids=squids, axis=2, ignore_missing=True)

The data contains 19 images of 128 x 128 pixels. Concatenation of multiple squids is performed using `axis=2`.

In [ ]:
print(np.shape(img))

In [ ]:
# information about the dataset
camera_info = data.dataset_attrs('IMG', squid)
camera_info

In [ ]:
# average image
av_img = np.mean(img, axis=2)
cmap = 'jet'

# plot
fig, ax = plt.subplots()
image = ax.imshow(av_img, cmap=cmap)
cb = plt.colorbar(image)

# output
plt.show()

## Dataframe datasets

Another common data storage format is the dataframe-like structure, i.e., single values stored as rows with headed columns.

In [ ]:
# load data from the hdf5 file,
df = data.df(dataset='AV_0', squids=data.squids, label=None, ignore_missing=False)
df.head()

In this example, AV_0 is the mean oscilloscope signal in a given window.  The window boundaries are stored as dataset attributes, here returned as `df_info`.

In [ ]:
df_info = data.dataset_attrs('AV_0', squid)
df_info

In [ ]:
# plot the DataFrame for squid=1
df.loc[1, :].plot(subplots=True)

# output
plt.show()

In [ ]:
# plot the DataFrame for all squids
df.reset_index()[['AB', 'CD']].plot(subplots=True)

# output
plt.show()

## H5Scan

These class methods are shared by `H5scan`, however, the argument `squid` is not needed as H5Scan is for use with files without groups.

In [ ]:
from e11 import H5Scan
fil = os.path.join(os.getcwd(), 'example_data', 'microwave_scan.h5')
scan = H5Scan(fil)

In [ ]:
arr2 = scan.array('osc_0')
osc0_info = scan.dataset_attrs('osc_0')

In [ ]:
# plot
fig, ax = plt.subplots()

# data
dt = 1e6 * osc0_info['dt']
xvals = np.arange(0, len(arr2[0])*dt, dt)
yvals = np.mean(arr2, axis=0)
ax.plot(xvals, yvals)

# format
ax.set_xlabel('time ($\mu$s)')
ax.set_ylabel('signal (V)')

# output
plt.show()

In [ ]:
df2 = scan.df('analysis')
df2.head()

In [ ]:
# plot the DataFrame for all squids
df2.sort_values('var').plot('var', 'f')

# output
plt.show()